In [ ]:
# bert
# "google-bert/bert-base-uncased"

# "google-bert/bert-large-uncased"

# roberta
# "FacebookAI/roberta-base"

# distilbert
# "distilbert-base-uncased"

In [ ]:
from transformers import AutoTokenizer, TFBertModel, TFRobertaModel
import tensorflow as tf

def bert_encode_text(X_train, X_val, X_test):
    tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-large-uncased", use_fast=True)
    model = TFBertModel.from_pretrained("google-bert/bert-large-uncased")

    print('Tokenizer is fast?', tokenizer.is_fast)

    def tkn(text):
        print('*')
        return tokenizer(text, return_tensors="tf", padding='max_length', truncation=True, max_length=35)

    def encode(X):
        batch_size = 200
        print('start')
        dataset = tf.data.Dataset.from_tensor_slices(X).batch(batch_size)
        print('dataset')
        pooled_outputs = []
        i = 0
        concatenated_output = None
        for batch in dataset:
            print('batch', i)
            #print(batch.numpy().tolist())
            texts = [str(text.numpy(), 'utf-8') for text in batch]
            #print(texts)
            inputs = tkn(texts)
            print('tkn')
            outputs = model(inputs)
            print('call')
            pooled_output = outputs['pooler_output']
            print('pool')
            pooled_outputs.append(pooled_output)
            #print(pooled_outputs)
            i += 1
            concatenated_output = tf.concat(pooled_outputs, axis=0)
            print('Concatenated output shape:', concatenated_output.shape)
        return concatenated_output


    return encode(X_train), encode(X_val), encode(X_test)



In [ ]:
import pandas as pd

fname = '/content/processed_func_data.csv'
df = pd.read_csv(fname, dtype={'id':str})

In [ ]:
df.head()

,filename,id,conversation_line,emotion,speaker,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,...,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3,split
0,dia1utt1,00,"Alright , so I am back in high school , I am s...",neutral,Chandler,4.400937e-07,1.535803e-11,4.400783e-07,394.0,544.0,...,7.081784,3.463025,30.857378,-0.806271,0.000000,0.975238,0.806271,0.975238,1.781509,train
1,dia1utt2,01,"Oh , yeah . Had that dream .",neutral,All,2.221042e-07,8.684786e-11,2.220173e-07,33.0,123.0,...,7.956037,2.999520,14.232584,-0.861038,-0.014966,4.652020,0.846072,4.666987,5.513059,train
2,dia1utt3,02,"Then I look down , and I realize there is a ph...",surprise,Chandler,7.866203e-07,8.263280e-12,7.866121e-07,491.0,350.0,...,5.199627,1.446559,15.534596,-0.184716,0.000000,0.597115,0.184716,0.597115,0.781831,train
3,dia1utt4,03,Instead of ... ?,surprise,Joey,2.768903e-07,1.009953e-10,2.767894e-07,36.0,0.0,...,15.080271,2.021774,7.933546,-1.659432,0.000000,3.845143,1.659432,3.845143,5.504575,train
4,dia1utt5,04,That is right .,anger,Chandler,7.087501e-07,8.615216e-10,7.078886e-07,32.0,55.0,...,7.078720,3.280567,15.817056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,train


In [ ]:
def get_text_features(df):
    train_df = df[df['split'] == 'train']
    val_df = df[df['split'] == 'val']
    test_df = df[df['split'] == 'test']

    train_text = train_df['conversation_line'].to_list()
    val_text = val_df['conversation_line'].to_list()
    test_text = test_df['conversation_line'].to_list()

    return train_text, val_text, test_text

In [ ]:
X_train_text, X_val_text, X_test_text = get_text_features(df)

In [ ]:
#bert_train, _, _ = bert_encode_text(X_train_text[0:1000], ['skip'], ['skip'])
bert_train, bert_val, bert_test = bert_encode_text(X_train_text, X_val_text, X_test_text)
#bert_train, bert_val, bert_test = bert_encode_text(X_train_text, X_val_text, X_test_text)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Tokenizer is fast? True
start
dataset
batch 0
*
tkn
call
pool
Concatenated output shape: (200, 1024)
batch 1
*
tkn
call
pool
Concatenated output shape: (400, 1024)
batch 2
*
tkn
call
pool
Concatenated output shape: (600, 1024)
batch 3
*
tkn
call
pool
Concatenated output shape: (800, 1024)
batch 4
*
tkn
call
pool
Concatenated output shape: (1000, 1024)
batch 5
*
tkn
call
pool
Concatenated output shape: (1200, 1024)
batch 6
*
tkn
call
pool
Concatenated output shape: (1400, 1024)
batch 7
*
tkn
call
pool
Concatenated output shape: (1600, 1024)
batch 8
*
tkn
call
pool
Concatenated output shape: (1800, 1024)
batch 9
*
tkn
call
pool
Concatenated output shape: (2000, 1024)
batch 10
*
tkn
call
pool
Concatenated output shape: (2200, 1024)
batch 11
*
tkn
call
pool
Concatenated output shape: (2400, 1024)
batch 12
*
tkn
call
pool
Concatenated output shape: (2600, 1024)
batch 13
*
tkn
call
pool
Concatenated output shape: (2800, 1024)
batch 14
*
tkn
call
pool
Concatenated output shape: (3000, 1024)
b

In [ ]:
print(bert_train, bert_val, bert_test)

tf.Tensor(
[[ 0.9730088  -0.5705834   0.22175097 ... -0.758729   -0.15664005
  -0.7235961 ]], shape=(1, 1024), dtype=float32) None None


In [ ]:
import numpy as np

np.save('bert_large_train_35', bert_train.numpy())
np.save('bert_large_val_35', bert_val.numpy())
np.save('bert_large_test_35', bert_test.numpy())

In [ ]:
t = np.load('/content/bert_train.npy')


array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [ ]:
from google.colab import files
files.download('/content/bert_large_train_35.npy')
files.download('/content/bert_large_val_35.npy')
files.download('/content/bert_large_test_35.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>